# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [14]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download(['punkt', 'wordnet'])

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#imblearn
from imblearn.over_sampling import RandomOverSampler

from sklearn.base import BaseEstimator, TransformerMixin
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from imblearn.under_sampling import RandomUnderSampler

[nltk_data] Downloading package punkt to C:\Users\Kumar
[nltk_data]     Aditya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Kumar
[nltk_data]     Aditya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql_table("messages", con=engine)

In [17]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


based on this quick check most of the data is very imbalanced

In [18]:
X = df["message"]
y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [19]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\Kumar Aditya\AppData\Local\Temp\ipykernel_36140\3411073879.py:1: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


In [21]:
def tokenize(text):
    '''
    Receives text related data and processes it
    Args: text related data (columns)
    Returns: tokenized text
    '''
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text) 
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [22]:
def multi_tester(X, y):
    '''
    Function to create list of fitted models
    Args: training data X and y
    returns: list of the selected fitted models
    '''
    pipe_1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
    ])
    
    pipe_2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(ExtraTreesClassifier(random_state=42)))
    ])
    
    pipe_3 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))
    ])
    
    pipe_4 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
    ])
    
    pipe_5 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SVC(random_state=42)))
    ])
    
    pips = [pipe_1, pipe_2, pipe_3, pipe_4, pipe_5]
    pip_names = ['RandomForestClassifier', 'ExtraTreesClassifier', 'GradientBoostingClassifier', 
                 'AdaBoostClassifier', 'SVC']
    
    model_fits = []
    for i in range(len(pips)):
        print('Model: ', pip_names[i])
        print(pips[i].get_params())
        mdl = pips[i].fit(X, y)
        model_fits.append(mdl)
        
    return model_fits

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)

In [24]:
fitted_mdls = multi_tester(X_train, y_train)

Model:  RandomForestClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))], 'verbose': False, 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>), 'tfidf': TfidfTransformer(), 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': <function tokenize at 0x00000236B9D144A0>, 'vect__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smo

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Kumar Aditya/nltk_data'
    - 'c:\\Users\\Kumar Aditya\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Kumar Aditya\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Kumar Aditya\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Kumar Aditya\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


### 5. Test your models
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
target_names = y_train.columns.tolist()

In [33]:
def perf_report(model, X_test, y_test):
    '''
    Function to return model classification reports
    Input: Model list, and test data 
    Output: Prints the Classification report
    '''
    pip_names = ['RandomForestClassifier', 'ExtraTreesClassifier', 'GradientBoostingClassifier', 
             'AdaBoostClassifier', 'SVC']
    
    for i in range(len(model)):
        print('______________________________Model______________________________')
        print('______________________________', pip_names[i], '______________________________')
        y_pred = model[i].predict(X_test)
        print(classification_report(y_test, y_pred, target_names = target_names))

In [12]:
perf_report(fitted_mdls, X_test, y_test)

______________________________Model______________________________
______________________________ RandomForestClassifier ______________________________


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      6534
               request       0.86      0.36      0.50      1472
                 offer       0.00      0.00      0.00        38
           aid_related       0.74      0.53      0.62      3545
          medical_help       0.69      0.07      0.13       701
      medical_products       0.64      0.07      0.12       446
     search_and_rescue       0.59      0.07      0.13       226
              security       0.00      0.00      0.00       160
              military       0.62      0.06      0.11       267
                 water       0.86      0.21      0.34       543
                  food       0.83      0.38      0.52       965
               shelter       0.82      0.21      0.33       775
              clothing       0.82      0.07      0.13       127
                 money       0.75      0.03      0.06       191
        missing_people       1.00      

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      6534
               request       0.00      0.00      0.00      1472
                 offer       0.00      0.00      0.00        38
           aid_related       0.00      0.00      0.00      3545
          medical_help       0.00      0.00      0.00       701
      medical_products       0.00      0.00      0.00       446
     search_and_rescue       0.00      0.00      0.00       226
              security       0.00      0.00      0.00       160
              military       0.00      0.00      0.00       267
                 water       0.00      0.00      0.00       543
                  food       0.00      0.00      0.00       965
               shelter       0.00      0.00      0.00       775
              clothing       0.00      0.00      0.00       127
                 money       0.00      0.00      0.00       191
        missing_people       0.00      

-shops has very little label diversity so it became an edge case, I will drop it for the optimization

______________________________ RandomForestClassifier ______________________________


                           precision    recall  f1-score   support
                           
                           
             micro avg       0.80      0.44      0.57     27308
             
             
             macro avg       0.58      0.16      0.21     27308
             
             
          weighted avg       0.74      0.44      0.50     27308
          
          
           samples avg       0.65      0.42      0.46     27308
           

______________________________ ExtraTreesClassifier ______________________________


             micro avg       0.79      0.44      0.56     27308
             
             
             macro avg       0.53      0.15      0.21     27308
             
             
          weighted avg       0.71      0.44      0.49     27308
          
          
           samples avg       0.66      0.42      0.46     27308

______________________________ GradientBoostingClassifier ______________________________


             micro avg       0.76      0.57      0.65     27308
             
             
             macro avg       0.51      0.32      0.38     27308
             
             
          weighted avg       0.72      0.57      0.61     27308
          
          
           samples avg       0.65      0.50      0.52     27308
           
           
______________________________ AdaBoostClassifier ______________________________


             micro avg       0.77      0.58      0.66     27308
             
             
             macro avg       0.58      0.33      0.40     27308
             
             
          weighted avg       0.73      0.58      0.62     27308
          
          
           samples avg       0.63      0.50      0.51     27308
           
______________________________ SVC ______________________________


             micro avg       0.76      0.24      0.36     27308
             
             
             macro avg       0.02      0.03      0.02     27308
             
             
          weighted avg       0.18      0.24      0.21     27308
          
          
           samples avg       0.76      0.32      0.40     27308


### 6. Improve models based on poor target performance elimination
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.
Testing models after dropping poor predictors

In [25]:
# dropping the targets that had the word performances based on the classification report
targs_drop = ['offer', 'security', 'infrastructure_related', 'tools', 
              'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'fire', 'other_weather']
y_min = y.copy()
y_min.drop(targs_drop, axis = 1, inplace = True)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y_min, random_state = 42, test_size = 0.33)

In [30]:
import nltk
nltk.download('all')  # This will download all corpora and resources



fitted_mdls_min = multi_tester(X_train, y_train)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Kumar
[nltk_data]    |     Aditya\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to C:\Users\Kumar
[nltk_data]    |     Aditya\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Kumar
[nltk_data]    |     Aditya\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Kumar
[nltk_data]    |     Aditya\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |    

Model:  RandomForestClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))], 'verbose': False, 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>), 'tfidf': TfidfTransformer(), 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': <function tokenize at 0x00000236B9D144A0>, 'vect__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smo

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Model:  GradientBoostingClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=GradientBoostingClassifier(random_state=42)))], 'verbose': False, 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>), 'tfidf': TfidfTransformer(), 'clf': MultiOutputClassifier(estimator=GradientBoostingClassifier(random_state=42)), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': <function tokenize at 0x00000236B9D144A0>, 'vect__vocabulary': None, 'tfidf__norm': 'l2',

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Model:  AdaBoostClassifier
{'memory': None, 'steps': [('vect', CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))], 'verbose': False, 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>), 'tfidf': TfidfTransformer(), 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': <function tokenize at 0x00000236B9D144A0>, 'vect__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': Tr

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Model:  SVC
{'memory': None, 'steps': [('vect', CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=SVC(random_state=42)))], 'verbose': False, 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>), 'tfidf': TfidfTransformer(), 'clf': MultiOutputClassifier(estimator=SVC(random_state=42)), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 1), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': <function tokenize at 0x00000236B9D144A0>, 'vect__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__use

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [31]:
target_names = y_train.columns.tolist()

In [34]:
perf_report(fitted_mdls_min, X_test, y_test)

______________________________Model______________________________
______________________________ RandomForestClassifier ______________________________


c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _

                   precision    recall  f1-score   support

          related       0.80      0.97      0.88      6534
          request       0.89      0.42      0.57      1472
      aid_related       0.77      0.61      0.68      3545
     medical_help       0.65      0.04      0.08       701
 medical_products       0.77      0.06      0.11       446
search_and_rescue       0.80      0.04      0.07       226
         military       0.69      0.04      0.08       267
            water       0.87      0.22      0.35       543
             food       0.87      0.42      0.56       965
          shelter       0.84      0.25      0.39       775
         clothing       0.75      0.07      0.13       127
            money       0.80      0.02      0.04       191
   missing_people       1.00      0.01      0.02       104
         refugees       0.33      0.00      0.01       293
            death       0.90      0.09      0.16       406
        other_aid       0.61      0.01      0.02      1

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _

                   precision    recall  f1-score   support

          related       0.80      0.97      0.88      6534
          request       0.90      0.42      0.57      1472
      aid_related       0.78      0.63      0.70      3545
     medical_help       0.61      0.04      0.08       701
 medical_products       0.66      0.07      0.12       446
search_and_rescue       0.39      0.03      0.06       226
         military       0.57      0.06      0.11       267
            water       0.82      0.15      0.26       543
             food       0.88      0.31      0.46       965
          shelter       0.84      0.22      0.35       775
         clothing       0.60      0.07      0.13       127
            money       0.60      0.03      0.06       191
   missing_people       0.75      0.03      0.06       104
         refugees       0.09      0.00      0.01       293
            death       0.86      0.08      0.14       406
        other_aid       0.48      0.02      0.04      1

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _

                   precision    recall  f1-score   support

          related       0.80      0.97      0.88      6534
          request       0.83      0.53      0.65      1472
      aid_related       0.77      0.56      0.65      3545
     medical_help       0.64      0.23      0.33       701
 medical_products       0.58      0.26      0.36       446
search_and_rescue       0.27      0.17      0.21       226
         military       0.51      0.22      0.31       267
            water       0.75      0.66      0.70       543
             food       0.80      0.79      0.79       965
          shelter       0.79      0.54      0.64       775
         clothing       0.57      0.47      0.52       127
            money       0.47      0.25      0.32       191
   missing_people       0.39      0.28      0.32       104
         refugees       0.47      0.24      0.31       293
            death       0.71      0.51      0.59       406
        other_aid       0.58      0.09      0.16      1

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _

                   precision    recall  f1-score   support

          related       0.83      0.91      0.87      6534
          request       0.78      0.55      0.64      1472
      aid_related       0.74      0.60      0.66      3545
     medical_help       0.55      0.25      0.35       701
 medical_products       0.63      0.31      0.41       446
search_and_rescue       0.56      0.17      0.26       226
         military       0.55      0.30      0.39       267
            water       0.73      0.63      0.68       543
             food       0.80      0.68      0.73       965
          shelter       0.76      0.57      0.65       775
         clothing       0.65      0.40      0.50       127
            money       0.48      0.26      0.34       191
   missing_people       0.53      0.08      0.13       104
         refugees       0.60      0.29      0.39       293
            death       0.78      0.43      0.55       406
        other_aid       0.54      0.16      0.25      1

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _


______________________________ RandomForestClassifier ______________________________

                           precision    recall  f1-score   support 
                           
              micro avg       0.80      0.48      0.60     25330
              
              macro avg       0.72      0.22      0.30     25330
              
           weighted avg       0.78      0.48      0.53     25330
           
            samples avg       0.66      0.44      0.48     25330

______________________________ ExtraTreesClassifier ______________________________

        micro avg       0.79      0.46      0.59     25330
        
        macro avg       0.68      0.20      0.27     25330
        
     weighted avg       0.75      0.46      0.52     25330
     
      samples avg       0.65      0.43      0.47     25330    

______________________________ GradientBoostingClassifier ______________________________

        micro avg       0.78      0.61      0.68     25330
        
        macro avg       0.65      0.43      0.50     25330
        
     weighted avg       0.76      0.61      0.65     25330
     
      samples avg       0.66      0.52      0.54     25330
           
           
______________________________ AdaBoostClassifier ______________________________

        micro avg       0.77      0.61      0.69     25330
        
        macro avg       0.69      0.42      0.51     25330
        
     weighted avg       0.75      0.61      0.66     25330
     
      samples avg       0.64      0.51      0.53     25330
           
______________________________ SVC ______________________________

        micro avg       0.76      0.26      0.38     25330
        
        macro avg       0.03      0.04      0.03     25330
        
     weighted avg       0.19      0.26      0.22     25330
     
      samples avg       0.76      0.33      0.41     25330


### 7. Improve your model
Use grid search to find better parameters. 

I will work on my best performing model adaboost and using the reduced target data

In [35]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

In [36]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfid

In [37]:
parameters = {'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__random_state': [42],
             'clf__estimator__learning_rate': [0.5]} 

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 10,
                  refit = True, verbose = 1, return_train_score = True, n_jobs = -1)

In [38]:
cv

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__learning_rate': [0.5],
                         'clf__estimator__n_estimators': [50, 100],
                         'clf__estimator__random_state': [42],
                         'tfidf__use_idf': (True, False)},
             return_train_score=True, verbose=1)

### 8. Test selected model

In [39]:
# dropping the targets that had the word performances based on the classification report
targs_drop = ['offer', 'security', 'infrastructure_related', 'tools', 
              'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'fire', 'other_weather']
y_min = y.copy()
y_min.drop(targs_drop, axis = 1, inplace = True)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y_min, random_state = 42, test_size = 0.33)

In [41]:
best_ada = cv.fit(X_train, y_train)


print('Best model :', best_ada.best_score_)
print('Params :', best_ada.best_params_)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best model : 0.2849022393990585
Params : {'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 100, 'clf__estimator__random_state': 42, 'tfidf__use_idf': True}


In [42]:
y_pred = best_ada.predict(X_test)
print(classification_report(y_test, y_pred, target_names = target_names))

                   precision    recall  f1-score   support

          related       0.81      0.96      0.88      6534
          request       0.81      0.52      0.64      1472
      aid_related       0.77      0.59      0.67      3545
     medical_help       0.59      0.19      0.29       701
 medical_products       0.70      0.20      0.32       446
search_and_rescue       0.79      0.12      0.20       226
         military       0.62      0.24      0.35       267
            water       0.74      0.65      0.69       543
             food       0.82      0.69      0.75       965
          shelter       0.80      0.52      0.63       775
         clothing       0.76      0.40      0.53       127
            money       0.52      0.22      0.31       191
   missing_people       0.91      0.10      0.17       104
         refugees       0.62      0.20      0.31       293
            death       0.83      0.34      0.48       406
        other_aid       0.59      0.08      0.14      1

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _

### 9. Other Approaches


Custom estimators (inspired by: [repo](https://github.com/hnbezz/Portfolio_under_construction/blob/master/Disaster_Response_Pipeline/ML%20Pipeline%20Preparation.ipynb) )

In [43]:
class StartVerbExtractor(BaseEstimator, TransformerMixin):
    def start_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) != 0:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
        return 0


    def fit(self, X, y=None):
        return self
    

    def transform(self, X):
        X_tag = pd.Series(X).apply(self.start_verb)
        return pd.DataFrame(X_tag)

In [44]:
def get_text_len(data):
    return np.array([len(text) for text in data]).reshape(-1, 1)

In [45]:
# dropping the targets that had the word performances based on the classification report
targs_drop = ['offer', 'security', 'infrastructure_related', 'tools', 
              'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'fire', 'other_weather', 'other_aid']
y_min = y.copy()
y_min.drop(targs_drop, axis = 1, inplace = True)
target_names = y_min.columns.tolist()

In [46]:
#stratifying data
mlss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.33, random_state=42)

for train_index, test_index in mlss.split(X, y_min):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_min.values[train_index], y_min.values[test_index]

In [47]:
y_train = pd.DataFrame(y_train,columns=target_names)
y_test = pd.DataFrame(y_test,columns=target_names)

In [48]:
pipeline_2 = Pipeline([
    ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('best', TruncatedSVD()),
                ('tfidf', TfidfTransformer())])), 
        ('start_verb', StartVerbExtractor())])), 
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

In [49]:
pipeline_2.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)),
                                                   ('best', TruncatedSVD()),
                                                   ('tfidf',
                                                    TfidfTransformer())])),
                                  ('start_verb', StartVerbExtractor())])),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)),
                                                 ('best', TruncatedSVD()),
               

In [50]:
parameters = {'clf__estimator__n_estimators': [100, 200, 300], 
              'clf__estimator__random_state': [42],
             'clf__estimator__learning_rate': [0.1]} 

cv_2 = GridSearchCV(pipeline, param_grid = parameters, cv = 10,
                  refit = True, verbose = 1, return_train_score = True, n_jobs = -1)

In [51]:
cv_2

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000236B9D144A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__learning_rate': [0.1],
                         'clf__estimator__n_estimators': [100, 200, 300],
                         'clf__estimator__random_state': [42]},
             return_train_score=True, verbose=1)

In [52]:
best_ada_2 = cv_2.fit(X_train, y_train)


print('Best model :', best_ada_2.best_score_)
print('Params :', best_ada_2.best_params_)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best model : 0.2637091400100866
Params : {'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 300, 'clf__estimator__random_state': 42}


In [53]:
y_pred = best_ada_2.predict(X_test)
print(classification_report(y_test, y_pred, target_names = target_names))

                   precision    recall  f1-score   support

          related       0.79      0.98      0.88      6570
          request       0.82      0.49      0.61      1477
      aid_related       0.80      0.54      0.65      3584
     medical_help       0.67      0.14      0.23       688
 medical_products       0.72      0.19      0.30       433
search_and_rescue       0.63      0.11      0.19       239
         military       0.72      0.17      0.27       284
            water       0.79      0.66      0.72       552
             food       0.79      0.72      0.75       965
          shelter       0.86      0.46      0.60       764
         clothing       0.76      0.33      0.46       134
            money       0.74      0.13      0.22       199
   missing_people       0.89      0.16      0.28        98
         refugees       0.69      0.17      0.27       289
            death       0.79      0.38      0.52       394
        transport       0.81      0.16      0.27       

c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Kumar Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _

In [69]:
test_text = ['there is a storm and people are trapped']
test = cv_2.predict(test_text)
print(y_train.columns.values[(test.flatten()==1)])

['related' 'weather_related' 'storm']


That is a pretty cool prediction, let's try a few more

In [61]:
test_text = ['we are having an earthquake, buildings are destroyed, victims need clothes']
test = cv_2.predict(test_text)
print(y_train.columns.values[(test.flatten()==1)])

['related' 'request' 'aid_related' 'buildings' 'weather_related'
 'earthquake' 'direct_report']


In [68]:
test_text = ['We are at KF Marotiere 85, we have food and water shortage, please send food for us']
test = cv_2.predict(test_text)
print(y_train.columns.values[(test.flatten()==1)])

['related' 'request' 'aid_related' 'water' 'food' 'direct_report']


### 9. Export your model as a pickle file

In [57]:
pickle.dump(cv_2, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.